# This notebook shows how to delete an existing slice

## Configure the environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

## Create slice manager object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host,refresh_token=fabric_refresh_token, project_name='all', scope='all')

## Refresh the ID Token

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print("New Refresh Token: {}".format(fabric_refresh_token))
print("Stored new Refresh Token")
%store fabric_refresh_token

## Query existing Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

## Retrieve Slice ID

In [ ]:
#Name of slice to be deleted
TargetSliceName = "ExampleSlice"

#Search for Slice ID from Slice Name
slice_id = list(filter(lambda slices: slices.slice_name == TargetSliceName, slices))[0].slice_id

print(slice_id)

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(result))